<a href="https://colab.research.google.com/github/ryanhalabi/starcraft_super_resolution/blob/master/super_resolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import os
import re 
import keras
import cv2
import numpy as np
import uuid
import shutil
import os
from pathlib import Path
from google.colab import drive
import tensorflow as tf

%tensorflow_version 2.x
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


drive.mount('/content/drive')
print(os.getcwd())

Using TensorFlow backend.


TensorFlow is already loaded. Please restart the runtime to change versions.
Found GPU at: /device:GPU:0
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content


In [2]:
class Environment(object):

  def __init__(self):
    pwd = Path(os.getcwd())
    print(pwd)
    if not os.path.isdir(pwd / 'drive' / 'My Drive' / 'super_res'):
      # shutil.rmtree(pwd / 'drive' / 'My Drive' / 'super_res' )
      os.mkdir( pwd / 'drive' / 'My Drive' / 'super_res'   )
      os.mkdir( pwd / 'drive' / 'My Drive' / 'super_res' / 'source'   )
      os.mkdir( pwd / 'drive' / 'My Drive' / 'super_res' / 'models' )
      os.mkdir( pwd / 'drive' / 'My Drive' / 'super_res' / 'model_output' )

    self.path = pwd / 'drive' / 'My Drive' / 'super_res'

env = Environment()



def download_images(urls):
  for url in urls:
      unit = re.search( r"[/\d]([\w]*).png", url).group(1)
      file_name = str(env.path / 'source' / f"{unit}.png")
      with open(file_name,'wb+') as f:
          f.write(requests.get(url).content)


/content


In [0]:
class Image:

    def __init__(self, path, greyscale):
        self.path = path
        self.greyscale = greyscale
        self.read_type = cv2.IMREAD_GRAYSCALE if greyscale else cv2.IMREAD_COLOR

    @property
    def name(self):
        return re.search( r"/([\w]*).png", str(self.path)).group(1)

    def get_array(self, scale=1):
        array = cv2.imread(str(self.path) , self.read_type)
        if scale != 1:
            array = cv2.resize(array, (int(array.shape[0]/scale), int(array.shape[1]/scale)))
        if self.greyscale:
            array = np.expand_dims(array,2)
        return array



In [0]:

class Model:

    def __init__(self, name, scaling=5, channels=1, conv_size=9):
        self.name = name
        self.scaling = scaling
        self.channels = channels
        self.conv_size = conv_size

        # scaling x the pixels, (conv_size - scaling)/2 extra pixels get added on each side as padding from conv2dtranspose
        # we dont care about these pixels for comparing against target
        # we also dont care about the pixels influenced by these extra pixels
        # get rid of depad filter of size 
        # (conv_size - scaling + 1) - gets rid of pad pixels
        # (conv_size+1) - gets rid of pixels influenced by padded pixels
        # so get rid of 2*conv_size - scaling pixels total

        # ex, 5x, conv 9
        # 0 0 x x X x+y x+y Y y+w y+w W
        # 18 - 5 = 13 = gets rid of 6 pixels each side
        # im probably fucking this up since further conv layers increase the impact of pixels
        # oh well

        self.set_optimizer()

        if not os.path.isdir(env.path / 'model_output' / self.name):
            os.mkdir( env.path / 'model_output' / self.name )
        for file in os.listdir( env.path / 'model_output' / self.name):
          os.remove(  env.path / 'model_output' / self.name / file)
        

        if os.path.isfile(env.path / 'models' / f"{name}.hdf5"):
            self.model = keras.models.load_model( str(env.path / 'models' / f"{name}.hdf5"))
        else:


            inputs = keras.layers.Input(shape=(100, 100, self.channels))
            upscaler = keras.layers.Conv2DTranspose(self.channels, (self.conv_size, self.conv_size), strides=(self.scaling,self.scaling))(inputs)

            conv_1 = keras.layers.Conv2D(64, (9,9), strides=(1,1), padding='same', activation='relu')(upscaler)
            conv_2 = keras.layers.Conv2D(32, (1,1), strides=(1,1), padding='same', activation='relu')(conv_1)
            conv_3 = keras.layers.Conv2D(self.channels, (5,5), strides=(1,1), padding='same')(conv_2)
            
            add_layer = keras.layers.add([conv_3, upscaler])

            depad_filter_size = 2*self.conv_size - self.scaling
            depad_kernel = np.zeros([depad_filter_size, depad_filter_size, self.channels, self.channels])
            center = int(depad_kernel.shape[0]/2)
            for i in range(self.channels):
              depad_kernel[center, center,i,i] = 1

            depad_bias = np.zeros([self.channels])

            predictions = keras.layers.Conv2D(self.channels, (depad_filter_size, depad_filter_size), strides=(1,1),
                                            weights=[depad_kernel, depad_bias], trainable=False)(add_layer)
            # predictions = keras.layers.Conv2D(self.channels, (depad_filter_size, depad_filter_size), strides=(1,1))(add_layer)

            model = keras.Model(inputs=inputs, outputs=predictions)

            model.compile(self.optimizer, 'mean_squared_error')

            self.model = model


    def set_optimizer(self):
        self.optimizer = keras.optimizers.Adam()


    def save_model(self):
        self.model.save( str(env.path / 'models' / f"{self.model.name}.hdf5"))


In [0]:
class TrainSRModel:

    def __init__(self, model, save=False):
        self.model = model
        self.save = save

        self.set_up_model()

    def set_up_model(self):
        up_model = keras.Sequential()
        up_model.add(keras.layers.UpSampling2D(size=(self.model.scaling,self.model.scaling), interpolation='bilinear', input_shape=(None, None, self.model.channels)))
        self.up_model = up_model


    def train(self, images, epochs, save):
        train_images = images

        self.X = np.array([x.get_array(self.model.scaling) for x in train_images])

        y = [x.get_array() for x in train_images]
        padding = int((self.model.conv_size-1)/2)
        y = [x[padding:-padding, padding:-padding,:] for x in y]
        self.Y = np.array(y)

        self.save_initial(images)

        for i in range(epochs):
            self.model.model.fit(self.X, self.Y, epochs = save, verbose=1) 
            print(f"Epoch {i} Loss: {self.model.model.history.history['loss'][-1]}")
            self.predict(images, i)
        self.predict(images, '0_final')

        if self.save:
            self.model.save(f"{self.model.name}.hdf5")

    def predict(self, images, name):

        x = np.array([x.get_array(self.model.scaling) for x in images])
        image_names = [x.name for x in images]

        preds = self.model.model.predict(x)

        for j, pred in enumerate(preds):
            cv2.imwrite( str(env.path / 'model_output' / self.model.name / f"{image_names[j]}_{name}.png"), pred)
        return preds



    def save_initial(self, images):

        image_names = [x.name for x in images]

        y = [x.get_array() for x in images]
        padding = int((self.model.conv_size-1)/2)
        y = [x[padding:-padding, padding:-padding,:] for x in y]
        Y = np.array(y)

        for j, y in enumerate(Y):
            cv2.imwrite( str(env.path / 'model_output' / self.model.name / f"{image_names[j]}.png"), y)

        X = np.array([x.get_array(self.model.scaling) for x in images])

        up_samples = self.up_model.predict(X)
        padding = int((self.model.conv_size-1)/2)
        up_samples = [x[padding:-padding, padding:-padding,:] for x in up_samples]

        for j, up_sample in enumerate(up_samples):
            cv2.imwrite( str(env.path / 'model_output' / self.model.name / f"{image_names[j]}_0_upscaled.png"), up_sample)




    # def plot_filters(self):

    #     weights = self.model.weights
    #     for weight in weights:
    #         for i in range( weight.shape[2]):
    #             filter = weight[:,:,i,0]
    #             plt.plot(filter)
    #             plt.show()


In [0]:
image_urls = [
                "https://liquipedia.net/commons/images/9/9d/Marine.png",
                "https://liquipedia.net/commons/images/8/8a/Firebat.png",
                "https://liquipedia.net/commons/images/2/26/Medic.png",
                "https://liquipedia.net/commons/images/f/f7/Scv.png",
                "https://liquipedia.net/commons/images/a/ab/Ghost.png",
                # "https://liquipedia.net/commons/images/thumb/1/13/%24Academy.png/600px-%24Academy.png",
                # "https://liquipedia.net/commons/images/thumb/d/dd/%24Armory.png/600px-%24Armory.png",
                # "https://liquipedia.net/commons/images/thumb/d/df/%24Barracks.png/600px-%24Barracks.png",
                # "https://liquipedia.net/commons/images/thumb/e/e9/%24Bunker.png/600px-%24Bunker.png",
                # "https://liquipedia.net/commons/images/thumb/d/dc/%24Command_Center.png/600px-%24Command_Center.png",
                # "https://liquipedia.net/commons/images/thumb/1/1f/%24Comsat_Station.png/600px-%24Comsat_Station.png",
                # "https://liquipedia.net/commons/images/thumb/2/2b/%24Control_Tower.png/600px-%24Control_Tower.png",
                # "https://liquipedia.net/commons/images/thumb/0/04/%24Covert_Ops.png/600px-%24Covert_Ops.png",
                # "https://liquipedia.net/commons/images/thumb/4/41/%24Engineering_Bay.png/600px-%24Engineering_Bay.png",
                # "https://liquipedia.net/commons/images/thumb/3/36/%24Factory.png/600px-%24Factory.png",
                # "https://liquipedia.net/commons/images/thumb/0/0b/%24Machine_Shop.png/600px-%24Machine_Shop.png",
                # "https://liquipedia.net/commons/images/thumb/4/41/%24Missile_Turret.png/600px-%24Missile_Turret.png",
                # "https://liquipedia.net/commons/images/thumb/e/ed/%24Nuclear_Silo.png/600px-%24Nuclear_Silo.png",
                # "https://liquipedia.net/commons/images/thumb/7/7a/%24Physics_Lab.png/600px-%24Physics_Lab.png",
                # "https://liquipedia.net/commons/images/thumb/c/ce/%24Refinery.png/600px-%24Refinery.png",
                # "https://liquipedia.net/commons/images/thumb/2/25/%24Science_Facility.png/600px-%24Science_Facility.png",
                # "https://liquipedia.net/commons/images/thumb/2/24/%24Starport.png/600px-%24Starport.png",
                # "https://liquipedia.net/commons/images/thumb/c/c7/%24Supply_Depot.png/600px-%24Supply_Depot.png"
                ]

download_images(image_urls)


images = [Image( env.path / 'source' / x, greyscale=False) for x in os.listdir(env.path / "source")]
model = Model('test_color',channels=3)

mt = TrainSRModel(model, False)


x = np.array([x.get_array(mt.model.scaling) for x in images])
image_names = [x.name for x in images]

mt.train(images, epochs=500, save=50)


# import matplotlib.pyplot as plt
# plt.matshow(mt.model.model.layers[0].get_weights()[0][:,:,0,0])
# plt.show()


# import numpy as np
# x = images[0].get_array(4)
# y = mt.model.predict( np.array([x]))
# # cv2.imwrite( str(env.path / 'images' / f"Atest.png"), x)




















Epoch 1/50
5/5 [==============================] - 4s 824ms/step - loss: 3938.6978
Epoch 2/50
5/5 [==============================] - 0s 77ms/step - loss: 3573.4810
Epoch 3/50
5/5 [==============================] - 0s 76ms/step - loss: 3232.8442
Epoch 4/50
5/5 [==============================] - 0s 75ms/step - loss: 2871.5068
Epoch 5/50
5/5 [==============================] - 0s 75ms/step - loss: 2471.5435
Epoch 6/50
5/5 [==============================] - 0s 74ms/step - loss: 2029.4375
Epoch 7/50
5/5 [==============================] - 0s 76ms/step - loss: 1552.2122
Epoch 8/50
5/5 [==============================] - 0s 75ms/step - loss: 1069.4253
Epoch 9/50
5/5 [==============================] - 0s 75ms/step - loss: 645.7274
Epoch 10/50
5/5 [==============================] - 0s 74ms/step - loss: 395.1926
Epoch 11/50
5/5 [==============================] - 0s 77ms/step - loss: 460.4016
Epoch 12/50
5/5 [==============================] - 0s 74ms/step - loss: 748.6197
Epoch 13/50
5/5

In [0]:
# import matplotlib.pyplot as plt
# preds = mt.predict(images,'test')

# print(preds.shape)
# x = mt.predict(images, image_names)

# plt.imshow(x[0,:,:,:])


# for j, pred in enumerate(preds):
#   cv2.imwrite( str(env.path / 'data' / 'test_color' / f"{image_names[j]}.png"), y)


In [0]:
mt.model.model.history.history['loss'][-1]

In [0]:
input = keras.layers.Input(shape=(5, 5, 3))

depad_filter_size = 3
depad_kernel = np.zeros([depad_filter_size, depad_filter_size, 3, 3])
center = int(depad_kernel.shape[0]/2)

for i in range(3):
  depad_kernel[center, center,i,i] = 1

depad_bias = np.zeros([3])

out = keras.layers.Conv2D(3, (depad_filter_size, depad_filter_size), strides=(1,1),
                                weights=[depad_kernel, depad_bias], trainable=False)(input)


model = keras.Model(inputs=input, outputs=out)

model.compile(keras.optimizers.Adam(), 'mean_squared_error')

model.summary()

x = np.random.randn(1,5,5,3)
y = model.predict(x)
print((y - x[:,1:-1,1:-1,:]).sum())

print(x[0,:,:,0])
print(y[0,:,:,0])

print(x[0,:,:,1])
print(y[0,:,:,1])

